In [1]:
import pandas as pd
from twitter_bot_detection.helpers import log_running_time, extract_urls
from kedro.io import PickleLocalDataSet

In [12]:
users = pd.read_pickle('data/02_intermediate/users.pkl')

In [13]:
@log_running_time
def prepare_users(users: PickleLocalDataSet) -> PickleLocalDataSet:
    deprecated = ["utc_offset", "time_zone", "lang", "geo_enabled", "following", "follow_request_sent", "has_extended_profile", "notifications", "profile_location", "contributors_enabled", "profile_image_url", "profile_background_color", "profile_background_image_url", "profile_background_image_url_https", "profile_background_tile", "profile_link_color", "profile_sidebar_border_color", "profile_sidebar_fill_color", "profile_text_color", "profile_use_background_image", "is_translator", "is_translation_enabled", "translator_type"]
    
    #extracting date of last tweet
    users["last_status_date"] = pd.to_datetime(users["status"].apply(lambda x: None if pd.isnull(x) else x["created_at"]))
    
    #extracting expanded profile url and urls from the description 
    users[["profile_url", "description_urls"]] = users["entities"].apply(extract_urls)
    
    #removing old accounts with no tweets publicly available
    users = users[~users["last_status_date"].isnull()]

    users.drop(columns=(deprecated + ['id', 'status', 'entities', 'url']), inplace=True)

    return users

In [14]:
prepare_users(users).to_pickle('data/03_primary/users.pkl')

2020-03-09 16:38:19,576 - twitter_bot_detection.helpers - INFO - Running 'prepare_users' took 14.79 seconds


/home/eugene/anaconda3/envs/twitter-bot-detection/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
